In [1]:
import keras
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [32]:
import numpy as np
from keras.layers import Conv2D,Flatten,Dense,Activation,Add,BatchNormalization,MaxPooling2D,AveragePooling2D,Input,ZeroPadding2D
from keras.initializers import glorot_uniform
from keras.models import Model
import keras
from keras.regularizers import l2

In [3]:
(X_train,y_train),(X_test,y_test)=keras.datasets.cifar10.load_data()

In [33]:
def identity_block(X,f,filters):
    
    X_temp = X
    F1,F2 = filters
    
    X = Conv2D(F1,(f,f), strides=(1,1),padding='same',kernel_initializer=glorot_uniform(seed=1),kernel_regularizer=l2(1e-4))(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    
    X = Conv2D(F2,(f,f), strides=(1,1), padding="same", kernel_initializer=glorot_uniform(seed=1),kernel_regularizer=l2(1e-4))(X)
    X=BatchNormalization(axis=3)(X)
    
    X=Add()([X,X_temp])
    X=Activation('relu')(X)
    
    return X
    

In [34]:
def conv_block(X,f,filters,s):
    X_temp = X
    F1,F2 = filters
    
    X_temp=Conv2D(F2,(1,1), strides=(s,s), padding="same", kernel_initializer=glorot_uniform(seed=1),kernel_regularizer=l2(1e-4))(X_temp)
    X_temp=BatchNormalization(axis=3)(X_temp)
    
    X = Conv2D(F1,(f,f), strides=(s,s),padding='same',kernel_initializer=glorot_uniform(seed=1),kernel_regularizer=l2(1e-4))(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    
    X = Conv2D(F2,(f,f), strides=(1,1), padding="same", kernel_initializer=glorot_uniform(seed=1),kernel_regularizer=l2(1e-4))(X)
    X=BatchNormalization(axis=3)(X)
    
    X=Add()([X,X_temp])
    X=Activation('relu')(X)
    
    return X
    

In [35]:
def Model34(input_shape,classes):
    
    X_input=Input(input_shape)
    X=ZeroPadding2D((3,3))(X_input)
    X=Conv2D(64,(7,7), strides=(2,2), kernel_initializer=glorot_uniform(seed=1),kernel_regularizer=l2(1e-4))(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    X=MaxPooling2D((3,3),strides=(2,2))(X)
    
    X=conv_block(X,3,(64,64),1)
    X=identity_block(X,3,(64,64))
    X=identity_block(X,3,(64,64))
    
    X=conv_block(X,3,(128,128),2)
    X=identity_block(X,3,(128,128))
    X=identity_block(X,3,(128,128))
    X=identity_block(X,3,(128,128))
    
    X=conv_block(X,3,(256,256),2)
    X=identity_block(X,3,(256,256))
    X=identity_block(X,3,(256,256))
    X=identity_block(X,3,(256,256))
    X=identity_block(X,3,(256,256))
    X=identity_block(X,3,(256,256))
    
    X=conv_block(X,3,(512,512),2)
    X=identity_block(X,3,(512,512))
    X=identity_block(X,3,(512,512))
    
    
    X = AveragePooling2D((2, 2))(X)
    
    X = Flatten()(X)
    
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0),kernel_regularizer=l2(1e-4))(X)
    
    model=Model(inputs = X_input, outputs = X, name='ResNet34')
    
    return model

In [76]:
def ModelCustom(input_shape,classes):
    
    X_input=Input(input_shape)
    X=ZeroPadding2D((1,1))(X_input)
    X=Conv2D(64,(3,3), strides=(1,1), kernel_initializer=glorot_uniform(seed=1),kernel_regularizer=l2(1e-4))(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    #X=MaxPooling2D((3,3),strides=(2,2))(X)
    
    X=conv_block(X,3,(32,32),1)
    X=identity_block(X,3,(32,32))
    X=identity_block(X,3,(32,32))
    X=identity_block(X,3,(32,32))
    
    X=conv_block(X,3,(64,64),2)
    X=identity_block(X,3,(64,64))
    X=identity_block(X,3,(64,64))
    X=identity_block(X,3,(64,64))
    
    X=conv_block(X,3,(128,128),2)
    X=identity_block(X,3,(128,128))
    X=identity_block(X,3,(128,128))
    X=identity_block(X,3,(128,128))
    
    #X=conv_block(X,3,(256,256),2)
    #X=identity_block(X,3,(256,256))
    
    
    #X=conv_block(X,3,(512,512),2)    #commented to reduce params
    #X=identity_block(X,3,(512,512))  # commented out to reduce params
    
    
    X = AveragePooling2D((8, 8))(X)
    
    X = Flatten()(X)
    
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0),kernel_regularizer=l2(1e-4))(X)
    
    model=Model(inputs = X_input, outputs = X, name='ResNet34')
    
    return model

In [77]:
X_train.shape

(50000, 32, 32, 3)

In [78]:
y_train.shape

(50000, 10)

In [79]:
model = ModelCustom((32,32,3),10)

In [80]:
model.summary()

Model: "ResNet34"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_9 (ZeroPadding2D (None, 34, 34, 3)    0           input_9[0][0]                    
__________________________________________________________________________________________________
conv2d_212 (Conv2D)             (None, 32, 32, 64)   1792        zero_padding2d_9[0][0]           
__________________________________________________________________________________________________
batch_normalization_212 (BatchN (None, 32, 32, 64)   256         conv2d_212[0][0]                 
___________________________________________________________________________________________

In [81]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [59]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [61]:
y_train=y_train[:,:,1]

In [62]:
y_test=y_test[:,:,1]

In [82]:
image_gen= keras.preprocessing.image.ImageDataGenerator( 
    horizontal_flip=True,
    zca_epsilon=1e-06,
    fill_mode="nearest",
    width_shift_range=0.15,
    height_shift_range=0.15)

image_gen.fit(X_train)

In [84]:
model.fit_generator(image_gen.flow(X_train, y_train, batch_size=128),
                        validation_data=(X_test, y_test),
                        epochs=10,workers=4)

Epoch 1/10
140/391 [=========>....................] - ETA: 13:52 - loss: 1.6110 - accuracy: 0.4848

KeyboardInterrupt: 